<h1><b>Amazon Redshift</b></h1>

- Importando bibliotecas e inicializando variáveis do envs
- Conectando ao Data Warehouse Redshift
- Criando tabelas e populando com dados dos scripts SQL (1 a 4)
- Carregando dados externos para o Redshift Spectrum do Bucket S3
- Importando dados do Bucket S3 para o Redshift
- Criando views e views materializadas e testando a diferença de desempelho entre elas

Obs.:

- Para executar o notebook, é necessário ter o Redshift configurado e rodando na AWS, além de ter criado um Bucket S3 para armazenar os dados. Os dois devem estar na mesma região da AWS.

- Os arquivos de exemplos estarão na pasta dados. Os dois scripts que serão utilizados para o Redshift Spectrum e a carga de dados devem estar em pastas separadas para evitar possíveis erros.

- O Cluster Redshift e o Bucket S3 devem estar configurados com as rules corretas e permissões para ser acessados publicamente.

Instalando bibliotecas utilizadas no notebook


In [5]:
%pip install psycopg2
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importando bibliotecas utilizadas no notebook e iniciando environment


In [11]:
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()

print(os.getenv('REDSHIFT_HOST'))

redshift-cluster.chflctapid3q.us-east-1.redshift.amazonaws.com


Inicializando variáveis utiilizandas no código com os valores do arquivo env


In [12]:
host = os.getenv('REDSHIFT_HOST')
database = os.getenv('REDSHIFT_DATABASE')
user = os.getenv('REDSHIFT_USER')
password = os.getenv('REDSHIFT_PASSWORD')
port = os.getenv('REDSHIFT_PORT')

role_iam = os.getenv('REDSHIFT_ROLE_IAM')
s3_bucket = os.getenv('REDSHIFT_S3_BUCKET')

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')

Conectando no Redshift


In [13]:
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

conn.autocommit = True
cur = conn.cursor()

<h2>Criação e Inserção dos dados no Data Warehouse Redshift</h2

Criação das tabelas no Redshift

In [14]:
cur.execute("CREATE TABLE categories (\
    category_id smallint NOT NULL,\
    category_name varchar(15) NOT NULL,\
    description VARCHAR(max)\
);")

cur.execute("CREATE TABLE customers (\
    customer_id varchar(40) NOT NULL,\
    company_name varchar(40) NOT NULL,\
    contact_name varchar(30),\
    contact_title varchar(30),\
    address varchar(60),\
    city varchar(15),\
    region varchar(15),\
    postal_code varchar(10),\
    country varchar(15),\
    phone varchar(24)\
);")

cur.execute("CREATE TABLE employees (\
    employee_id smallint NOT NULL,\
    last_name varchar(20) NOT NULL,\
    first_name varchar(10) NOT NULL,\
    title varchar(30),\
    title_of_courtesy varchar(25),\
    birth_date date,\
    hire_date date,\
    address varchar(60),\
    city varchar(15),\
    region varchar(15),\
    postal_code varchar(10),\
    country varchar(15),\
    home_phone varchar(24),\
    extension varchar(4),\
    notes varchar(max),\
    reports_to smallint,\
    photo_path varchar(255),\
	salary real\
);")

cur.execute("CREATE TABLE order_details (\
    order_id smallint NOT NULL,\
    product_id smallint NOT NULL,\
    unit_price real NOT NULL,\
    quantity smallint NOT NULL,\
    discount real NOT NULL\
);")

cur.execute("CREATE TABLE orders (\
    order_id smallint NOT NULL,\
    customer_id bpchar,\
    employee_id smallint,\
    order_date date DISTKEY SORTKEY,\
    required_date date,\
    shipped_date date,\
    ship_via smallint,\
    freight real,\
    ship_name varchar(40),\
    ship_address varchar(60),\
    ship_city varchar(15),\
    ship_region varchar(15),\
    ship_postal_code varchar(10),\
    ship_country varchar(15)\
) DISTSTYLE KEY;")

cur.execute("CREATE TABLE products (\
    product_id smallint NOT NULL,\
    product_name varchar(40) NOT NULL,\
    supplier_id smallint,\
    category_id smallint,\
    quantity_per_unit varchar(20),\
    unit_price real,\
    units_in_stock smallint,\
    units_on_order smallint,\
    reorder_level smallint,\
    discontinued integer NOT NULL\
);")

cur.execute("CREATE TABLE shippers (\
    shipper_id smallint NOT NULL,\
    company_name varchar(40) NOT NULL,\
    phone varchar(24)\
);")

cur.execute("CREATE TABLE suppliers (\
    supplier_id smallint NOT NULL,\
    company_name varchar(40) NOT NULL,\
    contact_name varchar(30),\
    contact_title varchar(30),\
    address varchar(60),\
    city varchar(15),\
    region varchar(15),\
    postal_code varchar(10),\
    country varchar(15),\
    phone varchar(24),\
    fax varchar(24),\
    homepage varchar(max)\
);")

Verificando a existência das tabelas criadas

In [15]:
cur.execute(
    "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")

tables = cur.fetchall()

for table in tables:
    print(table[0])

suppliers
shippers
products
orders
order_details
employees
customers
categories


Inserindo de dados nas tabelas do Redshift através dos scripts SQL (1 a 4)


In [16]:
with open('1.categories_shippers_suppliers.sql', 'r') as file:
    query1 = file.read()

with open('2.customers_employees_products.sql', 'r') as file:
    query2 = file.read()

with open('3.orders.sql', 'r') as file:
    query3 = file.read()

with open('4.orderdetails.sql', 'r') as file:
    query4 = file.read()

cur.execute(query1)
cur.execute(query2)
cur.execute(query3)
cur.execute(query4)

In [17]:
cur.execute(
    "SELECT * FROM orders WHERE order_date BETWEEN '2022-01-01' AND '2022-12-31';")

tables = cur.fetchall()

for table in tables:
    print(table)

(10808, 'OLDWO                                                                                                                                                                                                                                                           ', 2, datetime.date(2022, 1, 1), datetime.date(2022, 1, 29), datetime.date(2022, 1, 9), 3, 45.53, 'Old World Delicatessen', '2743 Bering St.', 'Anchorage', 'AK', '99508', 'USA')
(10809, 'WELLI                                                                                                                                                                                                                                                           ', 7, datetime.date(2022, 1, 1), datetime.date(2022, 1, 29), datetime.date(2022, 1, 7), 1, 4.87, 'Wellington Importadora', 'Rua do Mercado, 12', 'Resende', 'SP', '08737-363', 'Brazil')
(10810, 'LAUGB                                                                                             

<h2>Carregando dados externos Redshift Spectrum (Bucket S3)</h2>

<h6>Obs.: A role IAM deve ser antes associada ao cluster Redshift.</h6>


In [18]:
cur.execute(f"create external schema schema_dev \
from data catalog \
database '{database}' \
iam_role '{role_iam}' \
create external database if not exists;")

In [20]:
cur.execute(f"create external table schema_dev.employees ( \
EmployeeID bigint, \
LastName varchar, \
FirstName varchar, \
Title varchar, \
TitleOfCourtesy varchar, \
BirthDate bigint, \
HireDate bigint, \
Address varchar, \
City  varchar, \
Region varchar, \
PostalCode varchar, \
Country varchar, \
HomePhone varchar, \
Extension bigint, \
Notes varchar, \
ReportsTo bigint, \
PhotoPath varchar, \
Salary FLOAT8) \
stored as parquet location '{s3_bucket}redshift/';")

Verificando a existencia da tabela externa employees

In [21]:
cur.execute("SELECT tablename FROM svv_external_tables;")

tables = cur.fetchall()

for table in tables:
    print(table[0])

employees


Consulta na tabela externa employees, criada apartir de um arquivo parquet do Bucket S3.

In [22]:
cur.execute(f"SELECT * FROM {database}.schema_dev.employees;")

tables = cur.fetchall()

for table in tables:
    print(table)

(1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', -664761600000000, 704678400000000, '507 - 20th Ave. E.Apt. 2A', 'Seattle', 'WA', '98122', 'USA', '(206) 555-9857', 5467, 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed "The Art of the Cold Call."  Nancy is a member of Toastmasters International.', 2, 'http://accweb/emmployees/davolio.bmp', 2954.55)
(2, 'Fuller', 'Andrew', 'Vice President, Sales', 'Dr.', -563846400000000, 713750400000000, '908 W. Capital Way', 'Tacoma', 'WA', '98401', 'USA', '(206) 555-9482', 3457, 'Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981.  He is fluent in French and Italian and reads German.  He joined the company as a sales representative, was promoted to sales manager i', 0, 'http://accweb/emmployees/fuller.bmp', 2254.49)
(3, 'Leverling', 'Janet', 'Sales Representative', 'Ms.', -200102400000000, 702086400000000, '722 Moss Bay Blvd.',

In [23]:
cur.execute(f"select e.FirstName + ' ' + e.LastName nome, \
sum(od.unit_price * od.quantity - od.discount) total \
from order_details od \
inner join orders o on o.order_id = od.order_id \
inner join schema_dev.employees e on e.employeeid = o.employee_id \
group by nome \
order by total desc;")

tables = cur.fetchall()

for table in tables:
    print(table)

('Margaret Peacock', 250187.45025301)
('Janet Leverling', 213051.299479961)
('Nancy Davolio', 202143.710308552)
('Andrew Fuller', 177749.260476589)
('Robert King', 141295.990102768)
('Laura Callahan', 133301.030065536)
('Anne Dodsworth', 82963.9998178482)
('Michael Suyama', 78198.0999336243)
('Steven Buchanan', 75567.7500033379)


<h2>Carga de dados (Bucket S3)</h2>


Criando a tabela employees que receberá os dados importados do Bucket S3

In [24]:
cur.execute("CREATE TABLE employees2 ( \
    employee_id smallint NOT NULL, \
    last_name varchar(20) NOT NULL, \
    first_name varchar(10) NOT NULL, \
    title varchar(30), \
    title_of_courtesy varchar(25), \
    birth_date date, \
    hire_date date, \
    address varchar(60), \
    city varchar(15), \
    region varchar(15), \
    postal_code varchar(10), \
    country varchar(15), \
    home_phone varchar(24), \
    extension varchar(4), \
    notes varchar(max), \
    reports_to smallint, \
    photo_path varchar(255), \
	salary real);")

Importando dados do arquivo CSV employees.csv para a tabela employees2 com o comando COPY

In [25]:
cur.execute(f"copy employees2 \
from '{s3_bucket}employees/employees2.csv' \
CREDENTIALS 'aws_access_key_id={aws_access_key_id};aws_secret_access_key={aws_secret_access_key}' \
delimiter ';' \
region '{aws_region}' \
IGNOREHEADER 1 \
DATEFORMAT AS 'YYYY-MM-DD HH:MI:SS' \
removequotes;")

Verificando dados da tabela employees2

In [26]:
cur.execute("select * from employees2;")

tables = cur.fetchall()

for table in tables:
    print(table)

(1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', datetime.date(1948, 12, 8), datetime.date(1992, 5, 1), '507 - 20th Ave. E.Apt. 2A', 'Seattle', 'WA', '98122', 'USA', '(206) 555-9857', '5467', 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed ""The Art of the Cold Call.""  Nancy is a member of Toastmasters International.', 2, 'http://accweb/emmployees/davolio.bmp', 2954.55)
(2, 'Fuller', 'Andrew', 'Vice President, Sales', 'Dr.', datetime.date(1952, 2, 19), datetime.date(1992, 8, 14), '908 W. Capital Way', 'Tacoma', 'WA', '98401', 'USA', '(206) 555-9482', '3457', 'Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981.  He is fluent in French and Italian and reads German.  He joined the company as a sales representative, was promoted to sales manager in January 1992 and to vice president of sales in March 1993.  Andrew is a member of the Sales Management Roundtable, the

<h2>Views e Views Materializadas no Redshift</h2>


Criando a view e view materializada

In [27]:
cur.execute("create view view_employee as \
select \
    e.employee_id, \
    e.first_name, \
    e.last_name, \
    extract(month from o.order_date) as month, \
    extract(year from o.order_date) as year, \
    sum(od.unit_price * od.quantity) as total_sales \
from \
    employees e \
join orders o on e.employee_id = o.employee_id \
join order_details od on o.order_id = od.order_id \
group by \
    e.employee_id, \
    e.first_name, \
    e.last_name, \
    month, \
    year;")

cur.execute("create materialized view view_employee_mat as \
select \
    e.employee_id, \
    e.first_name, \
    e.last_name, \
    extract(month from o.order_date) as month, \
    extract(year from o.order_date) as year, \
    sum(od.unit_price * od.quantity) as total_sales \
from \
    employees e \
join orders o on e.employee_id = o.employee_id \
join order_details od on o.order_id = od.order_id \
group by \
    e.employee_id, \
    e.first_name, \
    e.last_name, \
    month, \
    year;")

Analisando diferença de desempenho entre view normal e view materializada com o comando explain

In [28]:
cur.execute(f"explain select * from view_employee where employee_id = 7")

tables = cur.fetchall()

for table in tables:
    print(table)

('XN HashAggregate  (cost=49.97..50.00 rows=2 width=93)',)
('  ->  XN Hash Join DS_DIST_ALL_NONE  (cost=27.05..49.16 rows=54 width=93)',)
('        Hash Cond: ("outer".order_id = "inner".order_id)',)
('        ->  XN Hash Join DS_DIST_ALL_NONE  (cost=0.12..10.60 rows=5 width=89)',)
('              Hash Cond: ("outer".employee_id = "inner".employee_id)',)
('              ->  XN Seq Scan on orders o  (cost=0.00..10.38 rows=5 width=8)',)
('                    Filter: (employee_id = 7)',)
('              ->  XN Hash  (cost=0.11..0.11 rows=1 width=83)',)
('                    ->  XN Seq Scan on employees e  (cost=0.00..0.11 rows=1 width=83)',)
('                          Filter: (employee_id = 7)',)
('        ->  XN Hash  (cost=21.55..21.55 rows=2155 width=8)',)
('              ->  XN Seq Scan on order_details od  (cost=0.00..21.55 rows=2155 width=8)',)
('----- Tables missing statistics: employees, order_details, orders -----',)
('----- Update statistics by running the ANALYZE command on th

In [29]:
cur.execute(f"explain select * from view_employee_mat where employee_id = 7")

tables = cur.fetchall()

for table in tables:
    print(table)

('XN Seq Scan on mv_tbl__view_employee_mat__0 derived_table1  (cost=0.00..2.40 rows=21 width=38)',)
('  Filter: (grvar_1 = 7)',)


In [30]:
cur.close()
conn.close()